In [75]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

In [89]:
# Import eurovision data

eurovision = pd.read_csv(os.path.join("../","data", "eurovision_data.csv"))
eurovision.head(5)

,Unnamed: 0,event,host_city,host_country,year,round,participant_name,song,participant_country,running_order,total_points,rank_ordinal,rank,qualified,winner,participant_link,event_link
0,1,Helsinki 2007,Helsinki,Finland,2007,Semi-Final,Eric Papilaya,Get A Life - Get Alive,Austria,27.0,4.0,27th,27.0,False,False,https://eurovision.tv/participant/eric-papilaya,https://eurovision.tv/event/helsinki-2007
1,2,Helsinki 2007,Helsinki,Finland,2007,Semi-Final,The KMGu0027s,Love Power,Belgium,24.0,14.0,26th,26.0,False,False,https://eurovision.tv/participant/the-kmg-s,https://eurovision.tv/event/helsinki-2007
2,3,Helsinki 2007,Helsinki,Finland,2007,Semi-Final,Gerli Padar,Partners In Crime,Estonia,23.0,33.0,22nd,22.0,False,False,https://eurovision.tv/participant/gerli-padar,https://eurovision.tv/event/helsinki-2007
3,4,Helsinki 2007,Helsinki,Finland,2007,Semi-Final,Anonymous,Salvem El Mu00f3n,Andorra,21.0,80.0,12th,12.0,False,False,https://eurovision.tv/participant/anonymous,https://eurovision.tv/event/helsinki-2007
4,5,Helsinki 2007,Helsinki,Finland,2007,Semi-Final,Olivia Lewis,Vertigo,Malta,20.0,15.0,25th,25.0,False,False,https://eurovision.tv/participant/olivia-lewis,https://eurovision.tv/event/helsinki-2007


In [90]:
# Import world data

world = gpd.read_file(os.path.join("../","data", "World_Countries_Generalized.geojson"))
world.head(5)

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Length,SHAPE_Area,geometry
0,1,Afghanistan,AF,Afghanistan,AF,50.803210,62.566714,"MULTIPOLYGON (((61.27655 35.60725, 61.29638 35..."
1,2,Albania,AL,Albania,AL,9.625038,3.073540,"MULTIPOLYGON (((19.57083 41.68527, 19.58195 41..."
2,3,Algeria,DZ,Algeria,DZ,70.540082,213.513461,"MULTIPOLYGON (((4.60335 36.88791, 4.63555 36.8..."
3,4,American Samoa,AS,United States,US,0.600124,0.013720,"MULTIPOLYGON (((-170.74390 -14.37555, -170.749..."
4,5,Andorra,AD,Andorra,AD,0.937352,0.055585,"MULTIPOLYGON (((1.44584 42.60194, 1.48653 42.6..."


In [103]:
# Filter to final 1st and 2nd winner, summarise by count

winner = eurovision[(eurovision['round']=='Final') & (eurovision['rank']==1)][["year","participant_country","song","total_points"]].sort_values(by = 'total_points',ascending = False)
winner.head(5)
runnerup = eurovision[(eurovision['round']=='Final') & (eurovision['rank']==2)][["year","participant_country","song","total_points"]].sort_values(by = 'total_points',ascending = False)
runnerup.head(5)

,year,participant_country,song,total_points
732,2017,Bulgaria,Beautiful Mess,615.0
744,2016,Australia,Sound Of Silence,511.0
630,2021,France,Voilu00e0,499.0
661,2019,Italy,Soldi,472.0
604,2022,United Kingdom,SPACE MAN,466.0


In [107]:
# Filter to final 1st and 2nd winner, summarise by count
winner_counts = pd.DataFrame(winner['participant_country'].value_counts())
winner_counts

,count
participant_country,
Ireland,7
Sweden,6
Netherlands,5
Luxembourg,5
France,5
United Kingdom,5
Israel,4
Denmark,3
Ukraine,3


In [108]:
runnerup_counts = pd.DataFrame(runnerup['participant_country'].value_counts())
runnerup_counts

,count
participant_country,
United Kingdom,16
France,7
Germany,6
Italy,5
Ireland,4
Belgium,4
Russia,4
Switzerland,4
Spain,4


In [120]:
winner_runnerup = winner_counts.merge(runnerup_counts, on = 'participant_country',how='outer',)
winner_runnerup.rename(columns = {'count_x':'wins','count_y':'runnerups'}, inplace = True)
winner_runnerup = winner_runnerup.replace(np.nan, 0)

winner_runnerup.head(5)

,wins,runnerups
participant_country,,
Ireland,7.0,4.0
Sweden,6.0,1.0
Netherlands,5.0,3.0
Luxembourg,5.0,2.0
France,5.0,7.0


In [121]:
# New field = ratio
winner_runnerup["ratio"] = (winner_runnerup.runnerups / winner_runnerup.wins).round(2)
winner_runnerup

,wins,runnerups,ratio
participant_country,,,
Ireland,7.0,4.0,0.57
Sweden,6.0,1.0,0.17
Netherlands,5.0,3.0,0.60
Luxembourg,5.0,2.0,0.40
France,5.0,7.0,1.40
United Kingdom,5.0,16.0,3.20
Israel,4.0,2.0,0.50
Denmark,3.0,1.0,0.33
Ukraine,3.0,2.0,0.67


In [122]:
winner_runnerup['grouping'] = np.where(winner_runnerup.wins==0, "Runner-up but never won", "Runner-up and won at least once")
winner_runnerup

,wins,runnerups,ratio,grouping
participant_country,,,,
Ireland,7.0,4.0,0.57,Runner-up and won at least once
Sweden,6.0,1.0,0.17,Runner-up and won at least once
Netherlands,5.0,3.0,0.60,Runner-up and won at least once
Luxembourg,5.0,2.0,0.40,Runner-up and won at least once
France,5.0,7.0,1.40,Runner-up and won at least once
United Kingdom,5.0,16.0,3.20,Runner-up and won at least once
Israel,4.0,2.0,0.50,Runner-up and won at least once
Denmark,3.0,1.0,0.33,Runner-up and won at least once
Ukraine,3.0,2.0,0.67,Runner-up and won at least once


In [130]:
# Join spatial data
world_eurowins = world.merge(winner_runnerup,how='left', left_on ='COUNTRY', right_on='participant_country')
world_eurowins[world_eurowins['COUNTRY'] == 'Italy']

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,SHAPE_Length,SHAPE_Area,geometry,wins,runnerups,ratio,grouping
111,112,Italy,IT,Italy,IT,68.044301,33.055064,"MULTIPOLYGON (((12.75357 43.97100, 12.76472 43...",3.0,5.0,1.67,Runner-up and won at least once


In [132]:
# Map
world_eurowins.to_file(os.path.join("../","data", "world_eurowins.geojson"), driver="GeoJSON")  